In [131]:
import pandas as pd
from scipy import stats
import numpy as np
tabla = pd.read_csv('transcriptos_proteinas.csv')
tabla.shape

(1279, 31)

In [132]:
tabla.head(5)

,Unicidad,spotID,Locus,ProteinFunctionDescription,CervilCEMean,CervilORMean,LevovilxCervilCEMean,LevovilxCervilORMean,LevovilCEMean,LevovilORMean,...,LA0147ORMean,CervilMean,Levovil_x_CervilMean,LevovilMean,FerumMean,LA1420_x_FerumMean,LA1420Mean,PlovdivMean,LA0147_x_PlovdivMean,LA0147Mean
0,0,JX036,Solyc07g049530,1-aminocyclopropane-1-carboxylate oxidase,0.6,2.9,0.8,4.1,0.6,3.6,...,3.3,103.840044,98.839282,94.226176,46.366078,87.112471,178.019529,56.021997,107.990786,183.155389
1,1,JX036,Solyc02g036350,1-aminocyclopropane-1-carboxylate oxidase,0.6,2.9,0.8,4.1,0.6,3.6,...,3.3,808.307598,641.366389,8.622687,10.862235,32.769708,23.401085,8.094163,19.040740,20.463382
2,2,JX036,Solyc02g081190,1-aminocyclopropane-1-carboxylate oxidase 4,0.6,2.9,0.8,4.1,0.6,3.6,...,3.3,44.178753,124.580060,124.338181,53.309966,53.916256,62.359434,39.669939,55.298962,62.518666
3,3,JX036,Solyc12g005940,1-aminocyclopropane-1-carboxylate oxidase,0.6,2.9,0.8,4.1,0.6,3.6,...,3.3,1.176047,2.176191,8.902781,4.869144,2.952634,5.688137,0.290646,0.623757,3.944122
4,4,JX036,Solyc07g049550,1-aminocyclopropane-1-carboxylate oxidase,0.6,2.9,0.8,4.1,0.6,3.6,...,3.3,11.134913,18.255827,10.403891,6.906611,14.137624,18.440314,5.706033,16.860712,14.858015


In [133]:
#selecciono de la tabla, los features, para poder correr las correlaciones tranquila, y comodamente
#es interesante aclarar, que al utilizar la columna 'Unicidad' ya inequivocamente me refiero a cada locus asociado
#a su spot posible de manera univoca. ya que luego puedo mergerar con la tabla original con criterio 'Unicidad',
#restituyendo asi el locus posible.

In [134]:
proteinas_CE = tabla.loc[:,['Unicidad', 'Locus','CervilCEMean', 'LevovilxCervilCEMean', 'LevovilCEMean', 
'FerumCEMean','FerumxLA1420CEMean','LA1420CEMean','PlovdivCEMean','LA0147xPlovdivCEMean', 'LA0147CEMean']]

proteinas_OR = tabla.loc[:,['Unicidad', 'Locus','CervilORMean', 'LevovilxCervilORMean', 'LevovilORMean', 
'FerumORMean','FerumxLA1420ORMean','LA1420ORMean','PlovdivORMean','LA0147xPlovdivORMean', 'LA0147ORMean']]

transcriptos = tabla.loc[:,['Unicidad', 'Locus', 'CervilMean', 'Levovil_x_CervilMean', 'LevovilMean',
'FerumMean', 'LA1420_x_FerumMean', 'LA1420Mean', 'PlovdivMean','LA0147_x_PlovdivMean', 'LA0147Mean']]

print('dim prot_CE: ' , proteinas_CE.shape , 'dim prot_OR: ' , proteinas_OR.shape , 'dim transcriptos: ' , transcriptos.shape)

dim prot_CE:  (1279, 11) dim prot_OR:  (1279, 11) dim transcriptos:  (1279, 11)


In [135]:
#algo que habia quedado chequear, era si en el campo 'ProteinFunctionDescription' de la tabla, figura
#'not avialable', para asi modificar dicha anotacion
tabla[tabla['ProteinFunctionDescription']=='not avialable']

,Unicidad,spotID,Locus,ProteinFunctionDescription,CervilCEMean,CervilORMean,LevovilxCervilCEMean,LevovilxCervilORMean,LevovilCEMean,LevovilORMean,...,LA0147ORMean,CervilMean,Levovil_x_CervilMean,LevovilMean,FerumMean,LA1420_x_FerumMean,LA1420Mean,PlovdivMean,LA0147_x_PlovdivMean,LA0147Mean


In [136]:
#de esta manera se observa que hay 5 filas con esta leyenda.
#ahroa queda verificar si algun locus termina en numero distinto de 0(ya que muchos de esos casos no habian podido
#ser identificados previamente, y es problable que ni siqueira aparezcan en la tabla de trnascirptos original)
#lo voy a resolver de una manera rudimentaria, pero eficaz
lista=[]
for i in range(0,tabla.shape[0]):
    if((tabla.loc[i,'Locus'][-1]=='0')):
        lista.append(tabla.loc[i,'Locus'])

In [137]:
len(lista)==tabla.shape[0]#como son iguales, queire decir que TODO loci temrina en '0'

True

In [138]:
#vamos a chequear por que no corre la corelacion, aparnetemente aparece un string " chloroplast"

In [139]:
#vamos a usar la herramienta de pandas pd.Series.str.match(pat, case=True, flags=0, na=None)

In [140]:
proteinas_CE.columns

Index(['Unicidad', 'Locus', 'CervilCEMean', 'LevovilxCervilCEMean',
       'LevovilCEMean', 'FerumCEMean', 'FerumxLA1420CEMean', 'LA1420CEMean',
       'PlovdivCEMean', 'LA0147xPlovdivCEMean', 'LA0147CEMean'],
      dtype='object')

In [141]:
#proteinas_CE[proteinas_CE['CervilCEMean'].str.match(" chloroplast")]

In [142]:
#subsane el error a mano, voy a vovler a correr todo el c'odigo para chequear que este todo bien

In [143]:
tabla.shape[0]

1279

In [144]:
len(lista)

1279

In [145]:
#ahora que ya chequie todo puedo reordenar las tablas, y correr las correlaciones.

In [146]:
#como las correlaciones que a mi me interesan son con y sin hibridos, para cada locus,
#comparando entre los genotipos. primero voy a trasponer las tablas que corresponda,
#y subseleccionar solo a los parentales y trasponer, segun corresponda. hecho para cada estadio celular

In [147]:
proteinas_todos_CE = proteinas_CE.T
proteinas_parentales_CE = (proteinas_CE.loc[:,['Unicidad', 'Locus','CervilCEMean', 'LevovilCEMean', 'FerumCEMean',
'LA1420CEMean','PlovdivCEMean','LA0147CEMean']]).T
proteinas_todos_OR = proteinas_OR.T
proteinas_parentales_OR = (proteinas_OR.loc[:,['Unicidad', 'Locus','CervilORMean', 'LevovilORMean', 'FerumORMean',
'LA1420ORMean','PlovdivORMean','LA0147ORMean']]).T
transcriptos_todos = transcriptos.T
transcriptos_parentales = (transcriptos.loc[:,['Unicidad', 'Locus', 'CervilMean', 'LevovilMean','FerumMean',
'LA1420Mean', 'PlovdivMean','LA0147Mean']]).T

In [148]:
transcriptos_parentales.shape

(8, 1279)

In [149]:
proteinas_parentales_OR.shape

(8, 1279)

In [150]:
transcriptos_todos.shape

(11, 1279)

In [151]:
proteinas_todos_CE.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, Unicidad to LA0147CEMean
Columns: 1279 entries, 0 to 1278
dtypes: object(1279)
memory usage: 110.3+ KB


In [152]:
proteinas_todos_CE

,0,1,2,3,4,5,6,7,8,9,...,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278
Unicidad,0,1,2,3,4,5,6,7,8,9,...,1535,1536,1546,1550,1551,1552,1553,1554,1555,1557
Locus,Solyc07g049530,Solyc02g036350,Solyc02g081190,Solyc12g005940,Solyc07g049550,Solyc07g049530,Solyc02g036350,Solyc02g081190,Solyc12g005940,Solyc07g049550,...,Solyc01g111760,Solyc10g055670,Solyc03g095900,Solyc09g089680,Solyc09g010020,Solyc12g006380,Solyc09g089740,Solyc09g089690,Solyc09g089730,Solyc09g089580
CervilCEMean,0.6,0.6,0.6,0.6,0.6,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
LevovilxCervilCEMean,0.8,0.8,0.8,0.8,0.8,1.2,1.2,1.2,1.2,1.2,...,0.6,0.6,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
LevovilCEMean,0.6,0.6,0.6,0.6,0.6,0.3,0.3,0.3,0.3,0.3,...,0.6,0.6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
FerumCEMean,0.6,0.6,0.6,0.6,0.6,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
FerumxLA1420CEMean,0.5,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4,...,0.5,0.5,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
LA1420CEMean,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.6,0.6,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7
PlovdivCEMean,0.6,0.6,0.6,0.6,0.6,0.3,0.3,0.3,0.3,0.3,...,0.4,0.4,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
LA0147xPlovdivCEMean,0.5,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4,...,0.6,0.6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [153]:
#de esta observacion, veo que de fila 2 para abajo. de col 1 en adelante

In [154]:
#ahora estamos listos para armar los ciclos y correr las correlaciones

In [155]:
#correlaciones para todos los genotipos en CE
lista_corr__coef_todos_CE=[]
lista_corr_pval_todos_CE=[]
for i in range(0 , proteinas_todos_CE.shape[1]):
    corr = stats.spearmanr(pd.to_numeric(proteinas_todos_CE.iloc[2:,i]) , pd.to_numeric(transcriptos_todos.iloc[2:,i]) , nan_policy='omit')
    lista_corr__coef_todos_CE.append(corr.correlation)
    lista_corr_pval_todos_CE.append(corr.pvalue)
print('coefs todos CE: ' , len(lista_corr__coef_todos_CE) , 'pval todos CE: ' , len(lista_corr_pval_todos_CE))

/home/mariano/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


coefs todos CE:  1279 pval todos CE:  1279


In [156]:
len(lista_corr__coef_todos_CE)

1279

In [157]:
len(lista_corr_pval_todos_CE)

1279

In [158]:
proteinas_todos_CE.shape[1]#resuelto el problema de las dimensiones

1279

In [159]:
proteinas_todos_CE

,0,1,2,3,4,5,6,7,8,9,...,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278
Unicidad,0,1,2,3,4,5,6,7,8,9,...,1535,1536,1546,1550,1551,1552,1553,1554,1555,1557
Locus,Solyc07g049530,Solyc02g036350,Solyc02g081190,Solyc12g005940,Solyc07g049550,Solyc07g049530,Solyc02g036350,Solyc02g081190,Solyc12g005940,Solyc07g049550,...,Solyc01g111760,Solyc10g055670,Solyc03g095900,Solyc09g089680,Solyc09g010020,Solyc12g006380,Solyc09g089740,Solyc09g089690,Solyc09g089730,Solyc09g089580
CervilCEMean,0.6,0.6,0.6,0.6,0.6,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
LevovilxCervilCEMean,0.8,0.8,0.8,0.8,0.8,1.2,1.2,1.2,1.2,1.2,...,0.6,0.6,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
LevovilCEMean,0.6,0.6,0.6,0.6,0.6,0.3,0.3,0.3,0.3,0.3,...,0.6,0.6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
FerumCEMean,0.6,0.6,0.6,0.6,0.6,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
FerumxLA1420CEMean,0.5,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4,...,0.5,0.5,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
LA1420CEMean,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.6,0.6,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7
PlovdivCEMean,0.6,0.6,0.6,0.6,0.6,0.3,0.3,0.3,0.3,0.3,...,0.4,0.4,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
LA0147xPlovdivCEMean,0.5,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4,...,0.6,0.6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [160]:
#ahora corro las correlaciones para todos lso genotipos en OR
lista_corr__coef_todos_OR=[]
lista_corr_pval_todos_OR=[]
for i in range(0 , proteinas_todos_OR.shape[1]):
    corr = stats.spearmanr(pd.to_numeric(proteinas_todos_OR.iloc[2:,i]) , pd.to_numeric(transcriptos_todos.iloc[2:,i]) , nan_policy='omit')
    lista_corr__coef_todos_OR.append(corr.correlation)
    lista_corr_pval_todos_OR.append(corr.pvalue)
print('coefs todos OR: ' , len(lista_corr__coef_todos_OR) , 'pval todos OR: ' , len(lista_corr_pval_todos_OR))

/home/mariano/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


coefs todos OR:  1279 pval todos OR:  1279


In [162]:
#correlaciones para parentales en CE
lista_corr_coef_parentales_CE=[]
lista_corr_pval_parentales_CE=[]
for i in range(0 , proteinas_parentales_CE.shape[1]):
    corr = stats.spearmanr(pd.to_numeric(proteinas_parentales_CE.iloc[2:,i]) , pd.to_numeric(transcriptos_parentales.iloc[2:,i]) , nan_policy='omit')
    lista_corr_coef_parentales_CE.append(corr.correlation)
    lista_corr_pval_parentales_CE.append(corr.pvalue)
print('coefs parentales CE: ' , len(lista_corr_coef_parentales_CE) , 'pval parentales CE: ' , len(lista_corr_pval_parentales_CE))

coefs parentales CE:  1279 pval parentales CE:  1279


In [163]:
#correlaciones para parentales en OR
lista_corr_coef_parentales_OR=[]
lista_corr_pval_parentales_OR=[]
for i in range(0 , proteinas_parentales_OR.shape[1]):
    corr = stats.spearmanr(pd.to_numeric(proteinas_parentales_OR.iloc[2:,i]) , pd.to_numeric(transcriptos_parentales.iloc[2:,i]) , nan_policy='omit')
    lista_corr_coef_parentales_OR.append(corr.correlation)
    lista_corr_pval_parentales_OR.append(corr.pvalue)
print('coefs parentales OR: ' , len(lista_corr_coef_parentales_OR) , 'pval parentales OR: ' , len(lista_corr_pval_parentales_OR))

coefs parentales OR:  1279 pval parentales OR:  1279


In [ ]:
#ahora que tengo todas las listas de correlaciones, vamos a crear, y ordenar un df con las correlaciones

In [165]:
len(lista_corr_coef_parentales_OR)

1279

In [169]:
columnas_corr = ['Coef_todos_CE' , 'PVal_todos_CE' , 'Coef_parentales_CE' , 'PVal_parentales_CE',
'Coef_todos_OR' , 'PVal_todos_OR' , 'Coef_parentales_OR' , 'PVal_parentales_OR']

df_corr = pd.DataFrame([lista_corr__coef_todos_CE , lista_corr_pval_todos_CE , lista_corr_coef_parentales_CE ,
lista_corr_pval_parentales_CE , lista_corr__coef_todos_OR , lista_corr_pval_todos_OR , 
lista_corr_coef_parentales_OR , lista_corr_pval_parentales_OR], index=columnas_corr)
df_corr.shape

(8, 1279)

In [172]:
df = df_corr.T

In [173]:
df.head()

,Coef_todos_CE,PVal_todos_CE,Coef_parentales_CE,PVal_parentales_CE,Coef_todos_OR,PVal_todos_OR,Coef_parentales_OR,PVal_parentales_OR
0,-0.275241,0.473487,-0.392792,0.441113,-0.535570,0.137262,-0.771429,0.072397
1,0.009175,0.981311,-0.392792,0.441113,-0.652725,0.056677,-0.885714,0.018845
2,0.192669,0.619438,-0.130931,0.804726,-0.209207,0.589047,-0.314286,0.544093
3,-0.137620,0.724029,-0.392792,0.441113,-0.493728,0.176760,-0.314286,0.544093
4,-0.330289,0.385346,-0.654654,0.158302,-0.460255,0.212522,-0.942857,0.004805


In [174]:
#para verificar que este bien
print(lista_corr__coef_todos_CE[0] , lista_corr_pval_todos_CE[0])

-0.27524094128159016 0.4734867010262832


In [175]:
print(lista_corr__coef_todos_CE[4] , lista_corr_pval_todos_CE[4])

-0.3302891295379082 0.38534613412878366


In [ ]:
#con esto chequeo que el df qued'o bien

In [ ]:
#ahora le agrego las columnas del locus , y  unicidad

In [176]:
df['Locus'] = tabla['Locus']
df['Unicidad'] = tabla['Unicidad']
df.shape

(1279, 10)

In [177]:
df.columns

Index(['Coef_todos_CE', 'PVal_todos_CE', 'Coef_parentales_CE',
       'PVal_parentales_CE', 'Coef_todos_OR', 'PVal_todos_OR',
       'Coef_parentales_OR', 'PVal_parentales_OR', 'Locus', 'Unicidad'],
      dtype='object')

In [178]:
df.to_csv('TablaCorrelacionesTodosLosGenotipos_vs_Parentales.csv')

In [179]:
df

,Coef_todos_CE,PVal_todos_CE,Coef_parentales_CE,PVal_parentales_CE,Coef_todos_OR,PVal_todos_OR,Coef_parentales_OR,PVal_parentales_OR,Locus,Unicidad
0,-0.275241,0.473487,-0.392792,0.441113,-0.535570,0.137262,-0.771429,0.072397,Solyc07g049530,0
1,0.009175,0.981311,-0.392792,0.441113,-0.652725,0.056677,-0.885714,0.018845,Solyc02g036350,1
2,0.192669,0.619438,-0.130931,0.804726,-0.209207,0.589047,-0.314286,0.544093,Solyc02g081190,2
3,-0.137620,0.724029,-0.392792,0.441113,-0.493728,0.176760,-0.314286,0.544093,Solyc12g005940,3
4,-0.330289,0.385346,-0.654654,0.158302,-0.460255,0.212522,-0.942857,0.004805,Solyc07g049550,4
...,...,...,...,...,...,...,...,...,...,...
1274,-0.096625,0.804682,-0.091077,0.863763,-0.118661,0.761084,0.057977,0.913132,Solyc12g006380,1552
1275,0.316228,0.407084,0.212512,0.686031,-0.033903,0.930999,-0.202920,0.699798,Solyc09g089740,1553
1276,0.395285,0.292352,0.212512,0.686031,0.661112,0.052514,0.811679,0.049858,Solyc09g089690,1554
1277,-0.395285,0.292352,-0.394665,0.438739,-0.491596,0.178927,-0.695725,0.124789,Solyc09g089730,1555
